In [250]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import linregress
from itertools import product

pd.options.plotting.backend = "plotly"

In [251]:
# https://docs.screeps.com/api/#Creep
CREEP_TICKS_LIFE = 1500
MAX_BODY_SIZE = 50
MAX_DISTANCE = 50
DEFAULT_DISTANCE = 30

In [252]:
# Assuming one moving part for each carry part
actual_max_size = round(MAX_BODY_SIZE / 3)
work_parts = [*range(1, actual_max_size)]
carry_parts = [*range(1, actual_max_size)]
distances = [*range(5, MAX_DISTANCE, 5)]
distances

[5, 10, 15, 20, 25, 30, 35, 40, 45]

In [253]:
def harvested_tick(work):
    return work * 2

def capacity(carry):
    return carry * 50

def ticks_to_full(work, carry):
    return capacity(carry) / harvested_tick(work)

def round_trip(work, carry, distance):
    return (distance * 2) + ticks_to_full(work, carry) + 1

def body_cost(work, carry):
    move = carry
    return work * 100 + carry * 50 + move * 50

def profit(work, carry, distance):
    return round(CREEP_TICKS_LIFE / round_trip(work, carry, distance) * capacity(carry) - body_cost(work, carry))

def cost_benefit(work, carry, distance):
    return profit(work, carry, distance) / body_cost(work, carry)

In [254]:
cost_benefit(5, 5, 5)

9.417

In [255]:
cost_benefit_heatmap = list(map(lambda wc: [str(wc), wc[0], wc[1], wc[2], cost_benefit(*wc)], product(work_parts, carry_parts, distances)))

In [256]:
df = pd.DataFrame(cost_benefit_heatmap, columns=["body","work_parts", "carry_parts", "distance", "cost_benefit"])
df["ratio"] = df['carry_parts'] / df['work_parts']
df = df.sort_values(by="cost_benefit")
df['ratio'] = round(df['ratio'], 4)
df['cost_benefit'] = round(df['cost_benefit'], 4)
df

,body,work_parts,carry_parts,distance,cost_benefit,ratio
2168,"(16, 1, 45)",16,1,45,-0.5235,0.0625
2024,"(15, 1, 45)",15,1,45,-0.4944,0.0667
2167,"(16, 1, 40)",16,1,40,-0.4659,0.0625
1880,"(14, 1, 45)",14,1,45,-0.4613,0.0714
2023,"(15, 1, 40)",15,1,40,-0.4331,0.0667
...,...,...,...,...,...,...
747,"(6, 4, 5)",6,4,5,9.8430,0.6667
2097,"(15, 10, 5)",15,10,5,9.8432,0.6667
1197,"(9, 6, 5)",9,6,5,9.8433,0.6667
1647,"(12, 8, 5)",12,8,5,9.8435,0.6667


In [257]:
px.scatter(df, x="ratio", y="cost_benefit", color="distance")

In [258]:
best_by_distance = df.groupby(['distance']).agg({'cost_benefit': 'max'})
best_by_distance

,cost_benefit
distance,
5,9.8440
10,7.1678
15,5.7158
20,4.7681
25,4.0877
30,3.5704
35,3.1606
40,2.8265
45,2.5478


In [259]:
df_reg = df[df['cost_benefit'].isin(best_by_distance['cost_benefit'])]
df_reg = df_reg.sort_values('distance').copy()
df_reg

,body,work_parts,carry_parts,distance,cost_benefit,ratio
297,"(3, 2, 5)",3,2,5,9.8440,0.6667
1675,"(12, 11, 10)",12,11,10,7.1678,0.9167
1235,"(9, 10, 15)",9,10,15,5.7158,1.1111
939,"(7, 9, 20)",7,9,20,4.7681,1.2857
1264,"(9, 13, 25)",9,13,25,4.0877,1.4444
1274,"(9, 14, 30)",9,14,30,3.5704,1.5556
807,"(6, 10, 35)",6,10,35,3.1606,1.6667
817,"(6, 11, 40)",6,11,40,2.8265,1.8333
1142,"(8, 15, 45)",8,15,45,2.5478,1.8750


In [261]:
regression = linregress(df_reg['distance'], df_reg['ratio'])
regression

LinregressResult(slope=0.029880333333333335, intercept=0.6257916666666666, rvalue=0.9880428683528986, pvalue=6.083759873409091e-07, stderr=0.0017623290821100269, intercept_stderr=0.04958589856642907)

In [264]:
df_reg['distance'] * regression.slope + regression.intercept

297     0.775193
1675    0.924595
1235    1.073997
939     1.223398
1264    1.372800
1274    1.522202
807     1.671603
817     1.821005
1142    1.970407
Name: distance, dtype: float64

In [271]:
round(regression.slope, 1), round(regression.intercept, 1)

(32.7, -19.9)